In [1]:
import warnings
import pandas as pd
import amp_pd_peptide
from sklearn.linear_model import LinearRegression
import xgboost as xg
import pickle
import numpy as np
import os

warnings.simplefilter(action='ignore')

In [2]:
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV


from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
 
import lightgbm as lgb


lgbm = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=800,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11,random_state=1234)

GBoost = GradientBoostingRegressor(n_estimators=800, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =2345)

KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)


ENet = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=8911)

lass = Lasso(alpha =0.0005, random_state=3456)

# ENet = ElasticNetCV(random_state=8911)

lasso = LassoCV(random_state=3456)


# ridge=RidgeCV()


xgr1=xg.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=800,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =11314, nthread = -1,tree_method="gpu_hist")


xgr2 = xg.XGBRegressor(learning_rate=0.01,n_estimators=800,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, random_state=77777,
#                        seed=27,
                                     reg_alpha=0.00006,tree_method="gpu_hist")


# Read Data

In [3]:
train = pd.read_csv("/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv")
sup = pd.read_csv('/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv')

In [4]:
train = train.append(sup, ignore_index=True)

# Train Model

In [5]:
from mlxtend.regressor import StackingCVRegressor
model = {}
target = ["updrs_1", "updrs_2", "updrs_3"]

for u in target: 
    temp = train.dropna(subset=[u])  
    if u == 'updrs_3':
        temp = temp[temp[u] != 0]
        
    # Train data
    X = temp['visit_month']
    y = temp[u]    
    
    stackmodel = StackingCVRegressor(regressors=(lgbm,GBoost,ENet,KRR,xgr1,xgr2),
                            meta_regressor=lass, cv=5,
                            use_features_in_secondary=True,
                            store_train_meta_features=True,
                            shuffle=True,
                            random_state=89765)
    stackmodel.fit(X.values.reshape(-1, 1), y)
     
#     mse=mean_squared_error(np.log(y_pred),np.log(y),squared=False)
    
#     xgr=xg.XGBRegressor(colsample_bytree=0.5, gamma=0.05, 
#                              learning_rate=0.05, max_depth=4, 
#                              min_child_weight=1.7817, n_estimators=1000,
#                     reg_alpha=0.5, reg_lambda=0.85,
#                              subsample=0.5, silent=1,
#                              random_state =1123, nthread = -1,verbosity=1) #,tree_method="gpu_hist")
    
#     xgr.fit(X.values.reshape(-1, 1), y)
    
    filename= str(u)
    
    pickle.dump(stackmodel,open(filename, 'wb'))  

    model[u]=filename
    
    print("model for ",u,"saved in file name ",filename)  


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[LightGBM] [Warning] min_data_in_leaf is set=6, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=6
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.2319, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2319
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=11, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=11
[L

# Predict

In [6]:
train

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On
...,...,...,...,...,...,...,...,...
4833,65382_0,65382,0,NaN,NaN,0.0,NaN,NaN
4834,65405_0,65405,0,5.0,16.0,31.0,0.0,NaN
4835,65405_5,65405,5,NaN,NaN,57.0,NaN,NaN
4836,65530_0,65530,0,10.0,6.0,24.0,0.0,NaN


In [7]:
def get_predictions(my_train, model):
    os.chdir(r'/kaggle/working/')

    # Forecast
    my_train = my_train.fillna(0)
    
    target = ["updrs_1", "updrs_2", "updrs_3",'updrs_4']
    
    for u in target:
        
        # Here is where we will save the final results
        my_train['result_' + str(u)] = 0
  
        # Predict    
        X = my_train["visit_month"]
        
        # updrs_4 will have only 0's, so we update for others
        if u != 'updrs_4':
            stmodel = pickle.load(open(model[u], 'rb'))
            r1=stmodel.predict(X.values.reshape(-1, 1)) 
            my_train['result_' + str(u)] =np.round(r1)

        
    # Format for final submission
    result = pd.DataFrame()

    for m in [0, 6, 12, 24]:
        for u in [1, 2, 3,4]:

            temp = my_train[["visit_id", "result_updrs_" + str(u)]]
            temp["prediction_id"] = temp["visit_id"] + "_updrs_" + str(u) + "_plus_" + str(m) + "_months"
            temp["rating"] = temp["result_updrs_" + str(u)]
            temp = temp [['prediction_id', 'rating']]

            result = result.append(temp) 
            
    result = result.drop_duplicates(subset=['prediction_id', 'rating'])

    return result

# Run once to check results
get_predictions(train, model)

,prediction_id,rating
0,55_0_updrs_1_plus_0_months,6.0
1,55_3_updrs_1_plus_0_months,6.0
2,55_6_updrs_1_plus_0_months,6.0
3,55_9_updrs_1_plus_0_months,6.0
4,55_12_updrs_1_plus_0_months,6.0
...,...,...
4833,65382_0_updrs_4_plus_24_months,0.0
4834,65405_0_updrs_4_plus_24_months,0.0
4835,65405_5_updrs_4_plus_24_months,0.0
4836,65530_0_updrs_4_plus_24_months,0.0


# Default Code to Submit Answers

In [8]:
from platform import python_version

if int(python_version().split(sep='.')[1]) >= 10:
    import amp_pd_peptide_310  
else:
    import amp_pd_peptide
    
amp_pd_peptide.make_env.func_dict['__called__'] = False
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()  

In [9]:
for (test, _, _, _) in iter_test:
        
    result = get_predictions(test, model)
    print(result)
    env.predict(result)   # register your predictions

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
                    prediction_id  rating
0    3342_0_updrs_1_plus_0_months     6.0
4   50423_0_updrs_1_plus_0_months     6.0
0    3342_0_updrs_2_plus_0_months     6.0
4   50423_0_updrs_2_plus_0_months     6.0
0    3342_0_updrs_3_plus_0_months    20.0
4   50423_0_updrs_3_plus_0_months    20.0
0    3342_0_updrs_4_plus_0_months     0.0
4   50423_0_updrs_4_plus_0_months     0.0
0    3342_0_updrs_1_plus_6_months     6.0
4   50423_0_updrs_1_plus_6_months     6.0
0    3342_0_updrs_2_plus_6_months     6.0
4   50423_0_updrs_2_plus_6_months     6.0
0    3342_0_updrs_3_plus_6_months    20.0
4   50423_0_updrs_3_plus_6_months    20.0
0    3342_0_updrs_4_plus_6_months     0.0
4   50423_0_updrs_4_plus_6_months     0.0
0   3342_0_updrs_1_plus_12_months     6.0
4  50423_0_updrs_1_plus_12_months     6.0
0   3342_0_updrs_2_plus_12_months     6.0
4  50423_0_updrs_2_plus_12_months  